In [1]:
import numpy as np 
import pandas as pd

In [2]:

file = "wifi_dataset.xlsx"

df_visitor = pd.read_excel(file, sheet_name="Visitor_Info")
df_session = pd.read_excel(file, sheet_name="Session_Info")
df_location = pd.read_excel(file, sheet_name="Location_Info")
df_visit = pd.read_excel(file, sheet_name="Visit_Info")
df_shop = pd.read_excel(file, sheet_name="Shop_Network_Info")



In [3]:
df_visitor

,device_id,mac_hash,device_type,os_version,device_vendor,connection_type,signal_strength
0,39e5b4830d4d,876c9b16254e,Android,8.4,Samsung,2.4GHz,-43
1,e443169117a1,fc82267b45dc,iOS,12.5,Samsung,2.4GHz,-89
2,56acd2f85500,29433eae6f7f,Tablet,9.3,Samsung,2.4GHz,-58
3,e162e30a5a4c,81efd660a281,Tablet,13.0,Samsung,5GHz,-38
4,0d9a5484f79e,41c0d64a0bbd,Tablet,15.0,Xiaomi,2.4GHz,-65
...,...,...,...,...,...,...,...
99995,0adf418ee29b,5f30174c8422,Tablet,9.0,Samsung,5GHz,-60
99996,48ba4008d1f7,25452669d19d,iOS,11.0,Samsung,2.4GHz,-59
99997,0d217544713c,deb125bde515,iOS,15.4,Samsung,5GHz,-86
99998,4fd45fb784eb,79228c576a39,Tablet,8.0,Apple,2.4GHz,-44


In [4]:
for name, df in {
    "Visitor_Info": df_visitor,
    "Session_Info": df_session,
    "Location_Info": df_location,
    "Visit_Info": df_visit,
    "Shop_Network_Info": df_shop
}.items():
    print(f"\n===== {name} =====")
    print(df.head())
    print(df.info())
    print(df.describe(include="all"))



===== Visitor_Info =====
      device_id      mac_hash device_type  os_version device_vendor  \
0  39e5b4830d4d  876c9b16254e     Android         8.4       Samsung   
1  e443169117a1  fc82267b45dc         iOS        12.5       Samsung   
2  56acd2f85500  29433eae6f7f      Tablet         9.3       Samsung   
3  e162e30a5a4c  81efd660a281      Tablet        13.0       Samsung   
4  0d9a5484f79e  41c0d64a0bbd      Tablet        15.0        Xiaomi   

  connection_type  signal_strength  
0          2.4GHz              -43  
1          2.4GHz              -89  
2          2.4GHz              -58  
3            5GHz              -38  
4          2.4GHz              -65  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   device_id        100000 non-null  object 
 1   mac_hash         100000 non-null  object 
 2   device_type      100000

In [5]:
df_session['entry_time'] = pd.to_datetime(df_session['entry_time'])
df_session['exit_time'] = pd.to_datetime(df_session['exit_time'])

df_visit['first_seen'] = pd.to_datetime(df_visit['first_seen'])
df_visit['last_seen'] = pd.to_datetime(df_visit['last_seen'])


In [6]:
df_visitor.isnull().sum()
df_session.isnull().sum()
df_location.isnull().sum()
df_visit.isnull().sum()
df_shop.isnull().sum()


session_id               0
near_store           25097
bandwidth_used_mb        0
packets_sent             0
packets_received         0
latency_ms               0
connection_status        0
dtype: int64

In [7]:
df_visitor['device_id'].nunique()
df_session['session_id'].nunique()
df_session['device_id'].nunique()
df_location['session_id'].nunique()
df_shop['session_id'].nunique()


100000

In [8]:
df_visitor.duplicated().sum()
df_session.duplicated().sum()
df_location.duplicated().sum()
df_visit.duplicated().sum()
df_shop.duplicated().sum()


0

In [9]:
df_shop['near_store'].fillna("Anywhere")

0               Café
1        Electronics
2            Clothes
3           Anywhere
4           Anywhere
            ...     
99995        Clothes
99996    Electronics
99997           Café
99998           Café
99999        Clothes
Name: near_store, Length: 100000, dtype: object

In [10]:
df_shop.isnull().sum()


session_id               0
near_store           25097
bandwidth_used_mb        0
packets_sent             0
packets_received         0
latency_ms               0
connection_status        0
dtype: int64

In [11]:
df_shop[df_shop['near_store'].isna()].head()


,session_id,near_store,bandwidth_used_mb,packets_sent,packets_received,latency_ms,connection_status
3,S000004,NaN,32.34,28137,23999,3.52,dropped
4,S000005,NaN,39.23,49896,31415,145.20,dropped
11,S000012,NaN,417.82,18224,395,89.97,dropped
12,S000013,NaN,300.54,15756,47613,35.80,dropped
16,S000017,NaN,474.10,44852,18450,130.84,connected


In [12]:
df_shop['near_store'].value_counts(dropna=False)


near_store
Café           25226
NaN            25097
Clothes        24898
Electronics    24779
Name: count, dtype: int64

In [13]:
df_shop['near_store'] = df_shop['near_store'].fillna("Anywhere")


In [14]:
df_shop['near_store'].isna().sum()


0

In [15]:
pip install pyodbc


Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install sqlalchemy


In [17]:
import sqlalchemy as sa
import urllib

server = r"DESKTOP-LVGOFK8"      
database = "wifi_db"             
driver = "ODBC Driver 17 for SQL Server"

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
connection_string = urllib.parse.quote_plus(connection_string)

engine = sa.create_engine(f"mssql+pyodbc:///?odbc_connect={connection_string}")


In [18]:
with engine.connect() as conn:
    query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'"
    result = conn.execute(sa.text(query))
    tables = result.fetchall()
    print(tables)

[('sysdiagrams',), ('visit_info',), ('location_info',), ('visitor_info',), ('session_info',), ('shop_network',)]


In [19]:
df_visitor.to_sql(
    name='visitor_info',
    con=engine,
    if_exists='replace',   
    index=False
)


134

In [20]:
df_session.to_sql(
    'session_info',
    engine,
    if_exists='replace',
    index=False
)



186

In [21]:
df_shop.to_sql(
    'shop_network',
    engine,
    if_exists='replace',
    index=False
)


134

In [22]:
df_visit.to_sql(
    'visit_info',
    engine,
    if_exists='replace',
    index=False
)


440

In [23]:
df_location.to_sql(
    'location_info',
    engine,
    if_exists='replace',
    index=False
)


186

In [24]:
df_visit.columns = df_visit.columns.str.strip()

df_visit['first_seen'] = pd.to_datetime(df_visit['first_seen'], errors='coerce').dt.date
df_visit['last_seen'] = pd.to_datetime(df_visit['last_seen'], errors='coerce').dt.date

In [ ]:
import pyodbc
import pandas as pd

def get_conn():
    return pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=DESKTOP-LVGOFK8;'   
        'DATABASE=wifi_db;'
        'Trusted_Connection=yes;'
    )

def run_query(sql):
    with get_conn() as conn:
        return pd.read_sql(sql, conn)

df = run_query("SELECT TOP 100 * FROM visit_info")
print(df.head())


C:\Users\TECH-PLANET\AppData\Local\Temp\ipykernel_17308\829847868.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, conn)


In [ ]:
q_avg_dwell = """
SELECT 
    AVG(DATEDIFF(MINUTE, entry_time, exit_time)) AS avg_dwell_minutes
FROM dbo.Session_Info
WHERE exit_time IS NOT NULL;
"""

q_peak_hours = """
SELECT TOP 3
    DATEPART(HOUR, entry_time) AS visit_hour,
    COUNT(*) AS total_visits
FROM dbo.Session_Info
GROUP BY DATEPART(HOUR, entry_time)
ORDER BY total_visits DESC;
"""

q_repeat_ratio = """
SELECT 
    COUNT(*) * 1.0 / (SELECT COUNT(DISTINCT device_id) FROM dbo.Visit_Info) AS repeat_visitor_ratio
FROM (
    SELECT device_id
    FROM dbo.Visit_Info
    WHERE total_visits > 1
) AS repeat_visitors;
"""

q_top_zones = """
SELECT TOP 10
    zone,
    COUNT(DISTINCT session_id) AS unique_visits
FROM dbo.Location_Info
GROUP BY zone
ORDER BY unique_visits DESC;
"""

q_top_visitors = """
SELECT TOP 10
    device_id,
    total_visits
FROM dbo.Visit_Info
ORDER BY total_visits DESC;
"""

q_common_paths = """
SELECT TOP 10
    movement_path,
    COUNT(*) AS path_count
FROM dbo.Location_Info
GROUP BY movement_path
ORDER BY path_count DESC;
"""

q_full_dwell = """
SELECT
    v.device_id,
    v.device_type,
    v.device_vendor,
    vi.total_visits,
    s.session_id,
    s.entry_time,
    s.exit_time,
    DATEDIFF(MINUTE, s.entry_time, s.exit_time) AS dwell_minutes,
    l.zone,
    l.movement_path,
    l.ap_id,
    n.near_store,
    n.bandwidth_used_mb,
    n.latency_ms,
    n.connection_status
FROM dbo.Visitor_Info v
JOIN dbo.Visit_Info vi
    ON v.device_id = vi.device_id
JOIN dbo.Session_Info s
    ON v.device_id = s.device_id
JOIN dbo.Location_Info l
    ON s.session_id = l.session_id
JOIN dbo.shop_network n
    ON s.session_id = n.session_id
ORDER BY v.device_id, s.entry_time;
"""

q_zone_disconnects = """
SELECT TOP 1
    l.zone,
    COUNT(*) AS disconnect_count
FROM dbo.Location_Info l
JOIN dbo.Session_Info s
    ON l.session_id = s.session_id
WHERE s.disconnects > 0
GROUP BY l.zone
ORDER BY disconnect_count DESC;
"""

q_zone_dwell = """
WITH ZoneDwell AS (
    SELECT
        l.zone,
        SUM(DATEDIFF(HOUR, s.entry_time, s.exit_time)) AS total_minutes
    FROM dbo.Session_Info s
    JOIN dbo.Location_Info l
        ON s.session_id = l.session_id
    GROUP BY l.zone
)
SELECT *
FROM ZoneDwell
ORDER BY total_minutes DESC;
"""


df_avg_dwell   = run_query(q_avg_dwell)
df_peak_hours  = run_query(q_peak_hours)
df_repeat_ratio = run_query(q_repeat_ratio)
df_top_zones   = run_query(q_top_zones)
df_top_visitors = run_query(q_top_visitors)
df_paths       = run_query(q_common_paths)
df_full_dwell  = run_query(q_full_dwell)
df_disconnects = run_query(q_zone_disconnects)
df_zone_time   = run_query(q_zone_dwell)


print(df_avg_dwell)
print(df_peak_hours)
print(df_zone_time.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_sql("SELECT AVG(DATEDIFF(MINUTE, entry_time, exit_time)) AS avg_dwell_minutes FROM dbo.Session_Info WHERE exit_time IS NOT NULL", conn)

value = df['avg_dwell_minutes'][0]

plt.figure(figsize=(4,3))
plt.text(0.5, 0.5, f"{value:.1f} min", fontsize=32, ha='center')
plt.title("Average Visitor Time inside the mall in minuits")
plt.axis('off')
plt.show()

In [ ]:
query = """
SELECT TOP 3
    DATEPART(HOUR, entry_time) AS visit_hour,
    COUNT(*) AS total_visits
FROM dbo.Session_Info
GROUP BY DATEPART(HOUR, entry_time)
ORDER BY total_visits DESC
"""

df = pd.read_sql(query, conn)

df = df.sort_values(by="total_visits", ascending=False)

plt.figure(figsize=(8,5))

bars = plt.bar(df['visit_hour'], df['total_visits'], color=['#4CAF50','#2196F3','#FF5722'])

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,
        height + 5,
        f"{int(height)}",
        ha='center',
        fontsize=11,
        fontweight='bold'
    )

plt.xlabel("Hour of Day")
plt.ylabel("Total Visits")
plt.title("Top 3 Busiest Hours")
plt.xticks(df['visit_hour'])
plt.ylim(0, df['total_visits'].max() * 1.2)  
plt.show()



In [ ]:
df = pd.read_sql("""
SELECT 
    SUM(CASE WHEN total_visits > 1 THEN 1 ELSE 0 END) AS repeat_visitors,
    COUNT(*) AS total_visitors
FROM dbo.Visit_Info
""", conn)

repeat = df['repeat_visitors'][0]
newv = df['total_visitors'][0] - repeat

plt.pie([repeat, newv], labels=['Repeat', 'New'], autopct='%1.1f%%')
plt.title("Repeat vs New Visitors")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd


df = pd.read_sql("""
SELECT TOP 10
    zone,
    COUNT(DISTINCT session_id) AS unique_visits
FROM dbo.Location_Info
GROUP BY zone
ORDER BY unique_visits DESC
""", conn)


plt.figure(figsize=(10,6))

bars = plt.barh(df['zone'], df['unique_visits'], height=0.55)

plt.gca().invert_yaxis()

for bar in bars:
    plt.text(
        bar.get_width() + max(df['unique_visits']) * 0.04,
        bar.get_y() + bar.get_height()/2,
        f"{bar.get_width():,}",   # formatted with thousands separator
        va='center',
        fontsize=10,
        fontweight='bold'
    )

plt.xlabel("count of visitors")
plt.title("Top 10 Busiest Zones ")
plt.grid(axis='x', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
df = pd.read_sql("""
SELECT TOP 10 movement_path, COUNT(*) AS path_count
FROM dbo.Location_Info
GROUP BY movement_path
ORDER BY path_count DESC
""", conn)

plt.figure(figsize=(8,4))
plt.barh(df['movement_path'], df['path_count'])
plt.gca().invert_yaxis()
plt.xlabel("Frequency")
plt.title("Top 10 Visitor Movement Paths")
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

query = """
SELECT 
    l.zone,
    COUNT(*) AS disconnect_count
FROM dbo.Location_Info l
JOIN dbo.Session_Info s
    ON l.session_id = s.session_id
WHERE s.disconnects > 0
GROUP BY l.zone
ORDER BY disconnect_count DESC;
"""

df = pd.read_sql(query, conn)

scaler = MinMaxScaler()
df["disconnect_norm"] = scaler.fit_transform(df[["disconnect_count"]])


pivot_df = df.pivot_table(
    values="disconnect_norm",
    index="zone",
    aggfunc="mean"
)

plt.figure(figsize=(8, 12))
sns.heatmap(
    pivot_df,
    cmap="coolwarm",         
    annot=True,
    fmt=".2f",              
    linewidths=0.5,
    cbar_kws={'label': 'Normalized Disconnects'}
)

plt.title("Disconnect Hotspots ( Heatmap)", fontsize=14)
plt.xlabel("")
plt.ylabel("Zone")
plt.tight_layout()
plt.show()
